# Market Forecasting

## LSTM & GRU for Stock Price Prediction

### Importation des librairies

In [3]:
import os
import yfinance as yf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import json, time, pickle
from PIL import Image as PILImage
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
from alpha_vantage.timeseries import TimeSeries 
from IPython.display import Image, display, clear_output
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.callbacks import Callback

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM, Dropout , GRU,Input,Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

ModuleNotFoundError: No module named 'yfinance'

### Constantes, loads, helpers élémentaires

In [ ]:
dailies='1min, 5min, 15min, 30min, 60min'.split(', ')
columns=["open","high","low","close","volume"]
models=dict()
histories=dict()
API_KEY=open("alpha_api_key.txt","r").read()
companies_data = json.load(open('companies_list.txt'))
companies_dict = {data["ticker"]: (data["name"], data["sector"], data["industry"]) for data in companies_data}
def plot_col(df,col):
    plt.figure(figsize=(16,3))
    plt.plot(df.index,df[col])
    plt.xlabel('Date')
    xticks,labels=range(0,len(df),312),list(df.index)[::312]
    plt.xticks(xticks,labels,rotation=30,fontsize=6)
    plt.ylabel(col)
    plt.title(f"Plot of {col}")
    plt.show()

def apply_split(df,date,value):
    for col in ['open','high','low','close']:
        df.loc[df.index < date, col] = df.loc[df.index < date, col] / value
    df.loc[df.index < date, 'volume'] = df.loc[df.index < date, 'volume'] * value
    return df

### Un callback pour l'arrêt des entrainements

In [ ]:
class Patience(Callback):
    def __init__(self, name,patience):
        super().__init__()
        self.name=name
        self.patience=patience
        self.min_val=float('inf')
        self.turns_without_improvement=0

    def on_epoch_end(self, epoch, logs=None):
        current_loss = logs.get(self.name)
        if current_loss is not None and current_loss < self.min_val:
            self.min_val=current_loss
            self.turns_without_improvement=0
        else:
            self.turns_without_improvement+=1
        if self.turns_without_improvement>self.patience:
            print(f"\nStopping training: loss did not decrease for {self.patience} epochs. Current {self.name}: {current_loss:.4f}")
            self.model.stop_training = True

### Une classe "Compagny" : 
- ticker, interval (par défaut daily), name, sector, industry et dates
- méthodes __repr__, __set_data__, __get_data__

In [ ]:
class Company:
    def __init__(self, ticker,interval='1D'):
        self.ticker=ticker
        self.interval=interval
        self.name,self.sector,self.industry = companies_dict[ticker]
        self.splits=yf.Ticker(self.ticker).splits
        self.splits={k.strftime("%Y-%m-%d"):v for k,v in self.splits.items()}
        self.dates=None
    def __repr__(self):
        return f"Company :{self.ticker} --> {self.name} ({self.sector}, {self.industry}, interval={self.interval})"
    def set_data(self):
        """period in '1d','w','1m', attention yfinance accès limité hors premium"""
        if self.interval not in dailies+['1D','1M']:
            print(f"interval not correct, select one in {dailies+['1D','1M']}")
            return
        file_name=f"{self.ticker}_{self.interval}.csv"
        ts = TimeSeries(key=API_KEY, output_format='pandas')
        if self.interval in dailies:
            data,metadata=ts.get_intraday(symbol=self.ticker, interval=self.interval,outputsize='full')
        elif self.interval=='1D':
            data,metadata=ts.get_daily(symbol=self.ticker, outputsize='full')
        elif self.interval=='1M':
            data,metadata=ts.get_monthly(symbol=self.ticker,outputsize='full')
        data.columns=columns
        data.index=data.index.strftime('%Y-%m-%d')
        data=data.iloc[::-1]
        for date in sorted(self.splits,reverse=True):
            if data.index[0]<date and date<=data.index[-1]:
                print(f"Applying unsplit at date {date} with value {self.splits[date]}")
                data=apply_split(data,date,self.splits[date])
        data.to_csv(file_name,index=True)
        self.dates=list(data.index)
        with open(f"{self.ticker}_{self.interval}_metadata.json", 'w') as f:
            json.dump(metadata, f, indent=4)
    def get_data(self):
        if self.interval not in dailies+['1D','1M']:
            print(f"interval not correct, select one in {dailies+['1D','1M']}")
            return None
        file_name=f"{self.ticker+'_'+self.interval}.csv"
        if file_name not in os.listdir():
            self.set_data()
        return pd.read_csv(file_name,index_col=0),json.load(open(f"{self.ticker+'_'+self.interval}_metadata.json"))
        

In [ ]:
goog=Company('GOOG')
print(goog)
goog.set_data()
df,metadata=goog.get_data()
print(f"metadata={metadata}")
df.head()

In [ ]:
df.describe().T

In [ ]:
plot_col(df,'close')

In [ ]:
plot_col(df,'volume')

### Affichage des moyennes mobiles : 6,30,60,90 jours

In [ ]:
W6 = df.rolling(window=6).mean()
W30 = df.rolling(window=30).mean()
W60 = df.rolling(window=60).mean()
W90 = df.rolling(window=90).mean()

In [ ]:
plt.figure(figsize=(15,4))
W6['close'][df.index>"2022"].plot(label='6 days rolling').autoscale(axis='x',tight=True)
W30['close'][df.index>"2022"].plot(label='30 days rolling').autoscale(axis='x',tight=True)
W60['close'][df.index>"2022"].plot(label='60 days rolling').autoscale(axis='x',tight=True)
W90['close'][df.index>"2022"].plot(label='90 days rolling').autoscale(axis='x',tight=True)
df['close'][df.index>"2022"].plot(label='Close Price',color='k',linestyle='-',linewidth=2).autoscale(axis='x',tight=True)
plt.legend()
plt.title('Moyennes mobiles depuis 2022',backgroundcolor='g',color='white',fontweight='bold')  
plt.xlabel('Date',labelpad=15)
plt.ylabel('Price',labelpad=15)
plt.grid(True)
plt.tight_layout()
plt.show()

### Standard scaling des données :
!['Les scalers'](scalers.png )


In [ ]:
scaler=MinMaxScaler()
scaled_data=scaler.fit_transform(df)
scaled=pd.DataFrame(scaled_data,columns=df.columns,index=df.index)
scaled.head()

### Train/Test split et préparation des données:
On prépare les données en spécifiant la taille de fenêtre servant la mémoire du modèle (window_size)

In [ ]:
train_size=round(len(scaled)*0.8)
val_size=round(train_size*0.75)
train,val,test_set=scaled[:val_size],scaled[val_size:train_size],scaled[train_size:]
print(f"train_size={val_size}, val_size={train_size-val_size}, test_size={len(scaled)-train_size}")

In [ ]:
def prepare_time_series_data(Data, window_size):
    data_array = Data.values
    sequences = np.array([data_array[i:i+window_size] for i in range(len(data_array) - window_size)])
    labels = data_array[window_size:]
    return sequences, labels

In [ ]:
window_size=60
X_all, y_all=prepare_time_series_data(scaled, window_size)
X_train, y_train = prepare_time_series_data(train, window_size)
X_val,y_val=prepare_time_series_data(val, window_size)
X_test, y_test = prepare_time_series_data(test_set, window_size)
print(f"shapes : X_train: {X_train.shape}, y_train: {y_train.shape}, X_val: {X_val.shape}, y_val: {y_val.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}")
print(f"X_all: {X_all.shape}, y_all: {y_all.shape}")

### Premier modèle LSTM simple:
- RNN + Dropout
- deux couches

In [ ]:
LSTM1 = Sequential()
LSTM1.add(Input(shape=(window_size,X_train.shape[2])))
LSTM1.add(LSTM(100,return_sequences=True))
LSTM1.add(Dropout(0.2))
LSTM1.add(LSTM(100,return_sequences=False))
LSTM1.add(Dropout(0.2))
LSTM1.add(Dense(X_train.shape[2]))
LSTM1.compile(optimizer='adam', loss='mean_squared_error',metrics=['mean_absolute_error'])

LSTM1.summary()

In [ ]:
%%time
early_stop=Patience('loss',3)
histories['LSTM1'] = LSTM1.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stop],verbose=1)
models['LSTM1'] = LSTM1
clear_output()
print()

In [ ]:
for key in histories['LSTM1'].history.keys():
    print(key," : ",histories["LSTM1"].history[key][-1])

In [ ]:
title=' Loss and Mean_absolute_error over Epochs '
xlabel=' Epochs '
LSTM1_losses = pd.DataFrame(LSTM1.history.history)
ax = LSTM1_losses.plot(figsize=(10,6),title=title)
ax.autoscale(axis='x',tight=True)
ax.set(xlabel=xlabel)

In [ ]:
LSTM1_losses.tail()

In [ ]:
def predict_and_inverse_transform(DF,model,scaler,start=None, stop=None):
    if start is None:start=0
    if stop is None:stop=len(DF)
    tranche=slice(start,stop,None)
    test_set=X_all[tranche]
    predictions = model.predict(test_set)
    inverse_predictions = scaler.inverse_transform(predictions)
    test_df = pd.DataFrame(inverse_predictions,columns=[col+'_pred' for col in DF.columns],index=DF.iloc[start+window_size:min(stop+window_size,len(df))].index)
    return test_df

In [ ]:
test_df_1_val = predict_and_inverse_transform(df, LSTM1, scaler,val_size,train_size)
test_df_1_test = predict_and_inverse_transform(df, LSTM1, scaler,train_size,len(df))
print(f"len(test_df_1_val)={len(test_df_1_val)}, len(test_df_1_test)={len(test_df_1_test)}")

In [ ]:
col='high'
col_index=df.columns.get_loc(col)
indexes=df.index.to_list()
start_plot_date=indexes[len(indexes)//2]
start_index=indexes.index(start_plot_date)
print(f"start_index={start_index}")
plt.figure(figsize=(16,4))
plt.plot(df.index[start_index:], df[col][start_index:], label='Observed', color='k')
plt.plot(test_df_1_val.index, test_df_1_val[col+'_pred'], label='Predicted - Validation',color='g')
plt.plot(test_df_1_test.index, test_df_1_test[col+'_pred'], label='Predicted - Test',color='m')
plt.legend()
plt.title(f"Prédictions val et test de {col}  --> Impressionnant!",backgroundcolor='b',alpha=0.5,color='white',fontweight='bold')  
plt.xlabel('Date',labelpad=15)
plt.ylabel('Price',labelpad=15)
plt.xticks(df.index[start_index::312],rotation=30,fontsize=6)
plt.grid(True)
plt.tight_layout()


### Deuxième LSTM :
- trois couches LSTM avec Dropout
- une couche dense

In [ ]:
LSTM2 = Sequential()

# First LSTM layer
LSTM2.add(Input(shape=(window_size,X_train.shape[2])))
LSTM2.add(LSTM(150, return_sequences=True))
LSTM2.add(Dropout(0.2))
# Second LSTM layer
LSTM2.add(LSTM(100, return_sequences=True))
LSTM2.add(Dropout(0.2))
# Third LSTM layer 
LSTM2.add(LSTM(100, return_sequences=False)) 
LSTM2.add(Dropout(0.2))
# Final Prediction (one neuron per feature)
LSTM2.add(Dense(units=50))
LSTM2.add(Dense(units=5))
LSTM2.add(Dense(X_train.shape[2]))

LSTM2.compile(optimizer='adam', loss='mean_squared_error',metrics=['mean_absolute_error'])
LSTM2.summary()

In [ ]:
%%time
early_stop = early_stop=Patience('loss',3)
histories['LSTM2'] = LSTM2.fit(X_train, y_train,epochs=30,validation_data=(X_val, y_val),batch_size = 32,callbacks=[early_stop],verbose=1)
models['LSTM2'] = LSTM2
clear_output()
print()

In [ ]:
test_df_2_test = predict_and_inverse_transform(df, LSTM2, scaler,train_size,len(df))
test_df_2_val = predict_and_inverse_transform(df, LSTM2, scaler,val_size,train_size)

### Modèle GRU : 
- une couche GRU
- une couche Dense

In [ ]:
GRU_Model = Sequential()
GRU_Model.add(Input(shape=(window_size, X_train.shape[2])))
GRU_Model.add(GRU(128, activation='tanh'))
GRU_Model.add(Dense(X_train.shape[2]))
GRU_Model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
GRU_Model.summary()

In [ ]:
%%time
early_stop = early_stop=Patience('loss',3)
histories['GRU'] = GRU_Model.fit(X_train, y_train, epochs=30,validation_data=(X_val, y_val),batch_size = 32,callbacks=[early_stop],verbose=1)
models['GRU'] = GRU_Model
clear_output()
print()

In [ ]:
test_df_3_test = predict_and_inverse_transform(df, GRU_Model, scaler,train_size,len(df))
test_df_3_val = predict_and_inverse_transform(df, GRU_Model, scaler,val_size,train_size)

### Bidirectionnal GRU:
- une couche Bidirectionnal sur GRU suivie de dropout

In [ ]:
GRU_bidirectional = Sequential()
GRU_bidirectional.add(Input(shape=(window_size, X_train.shape[2])))
GRU_bidirectional.add(Bidirectional(GRU(128, activation='tanh')))
GRU_bidirectional.add(Dropout(0.2))
GRU_bidirectional.add(Dense(X_train.shape[2]))

GRU_bidirectional.compile(optimizer='adam', loss='mean_squared_error',metrics=['mean_absolute_error'])
GRU_bidirectional.summary()

In [ ]:
%%time
early_stop =Patience('loss',3)
histories['GRU_bidirectional'] = GRU_bidirectional.fit(X_train, y_train, epochs=30,validation_data=(X_val, y_val),batch_size = 32,callbacks=[early_stop],verbose=1)
models['GRU_bidirectional'] = GRU_bidirectional
clear_output()
print()

In [ ]:
test_df_4_test = predict_and_inverse_transform(df, GRU_bidirectional, scaler,train_size,len(df))
test_df_4_val = predict_and_inverse_transform(df, GRU_bidirectional, scaler,val_size,train_size)

In [ ]:
col='close'
col_index=df.columns.get_loc(col)
indexes=df.index.to_list()
start_plot_date="2013-06-25"
start_index=indexes[len(indexes)//2]
print(f"start_index={start_index}")
plt.figure(figsize=(16,4))
plt.plot(df.index[len(indexes)//2:], df[col][start_index:], label='Obs', color='k')
plt.plot(test_df_1_val.index, test_df_1_val[col+'_pred'], label='Pred - Val LSTM1',)
plt.plot(test_df_1_test.index, test_df_1_test[col+'_pred'], label='Pred - Test LSTM1')
plt.plot(test_df_2_val.index, test_df_2_val[col+'_pred'], label='Pred - Val LSTM2',)
plt.plot(test_df_2_test.index, test_df_2_test[col+'_pred'], label='Pred - Test LSTM2')
plt.plot(test_df_3_val.index, test_df_3_val[col+'_pred'], label='Pred - Val GRU',)
plt.plot(test_df_3_test.index, test_df_3_test[col+'_pred'], label='Pred - Test GRU')
plt.plot(test_df_4_val.index, test_df_4_val[col+'_pred'], label='Pred - Val GRU_bid')
plt.plot(test_df_4_test.index, test_df_4_test[col+'_pred'], label='Pred - Test GRU_bid')

plt.legend()
plt.title(f"Comparison of Actual and Predicted {col} Daily Prices for LSTM1, LSTM2 and GRU, on validation and test datasets",
          backgroundcolor='b',alpha=0.5,color='white',fontweight='bold')  
plt.xlabel('Date',labelpad=15)
plt.ylabel('Price',labelpad=15)
plt.xticks(df.index[len(indexes)//2::312],rotation=30,fontsize=6)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
for name in histories:
    print(name)
    for k,v in histories[name].history.items():
        print(f"k={k}, v={v[-1]}")  

In [ ]:
metrics = ['mean_absolute_error', 'loss', 'val_mean_absolute_error', 'val_loss']
results={name:[histories[name].history[k][-1] for k in metrics] for name in histories}
x = np.arange(len(metrics))  
bar_width = 0.2
plt.figure(figsize=(16,4))
for i,(name,values) in enumerate(results.items()):
    plt.bar(x+i*bar_width, values, bar_width, label=name,alpha=0.8)
plt.xlabel('Metrics')
plt.ylabel('Values')
plt.title('Comparison of Models on Various Metrics')
plt.xticks(x + bar_width * (len(results) - 1) / 2, metrics)  
plt.legend(title="Models")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()